### RNN e Word Embedding con Keras
Le reti neurali ricorrenti (RNNs - Recurrent Neural Networks) sono un'architettura di reti neurali artificiali che mantenedo una memoria interna ci permettono di analizzare sequenze temporale di dati.
<br><br>
In questo costruiremo una rete neurale ricorrente per un problema di sentiment analysis, utilizzando sempre il dataset l'Internet Movie Database.
<br><br>
Carichiamo i moduli che ci serviranno.

In [0]:
import numpy as np
import matplotlib.pyplot as plt

from keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import Dense

## Scarichiamo il dataset
Utilizziamo Keras per caricare l'imdb dataset, limitandolo alle 10000 parole più comuni.

In [30]:
from keras.datasets import imdb 

num_words = 10000

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=num_words)

print("Numero di esempi nel train set: %d" % len(X_train))
print("Numero di esempi nel test set: %d" % len(X_test))

Numero di esempi nel train set: 25000
Numero di esempi nel test set: 25000


## Preprocessiamo i dati
Per rappresentare le nostre sequenze utilizzeremo il Word Embedding, che va aggiunto come strato iniziale della nostra rete neurale.
Le recensioni all'interno del corpus di testo hanno ovviamente lunghezza differente


In [31]:
longest_review = max(X_train,key=len)
shortest_review = min(X_train,key=len)

print("La review più lunga ha %d parole" % len(longest_review))
print("La review più corta ha %d parole" % len(shortest_review))

La review più lunga ha 2494 parole
La review più corta ha 11 parole


trochiamole dopo 100 parole utilizzando la funzione pad_sequence di keras. Se una recensione ha meno di 100 parole verranno aggiunti una serie di zeri al termine per portarla alla lunghezza corretta.

In [32]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 100

X_train = pad_sequences(X_train, maxlen = maxlen)
X_test = pad_sequences(X_test, maxlen = maxlen)

X_train.shape

(25000, 100)

## Creiamo il modello
Il primo strato del nostro modello eseguirà il Word Embedding. Il Word embedding è un modello che ci permette di generare una serie di vettori (embedding vectors) ognuno dei quali quantifica una caratteristica delle parole. Questo tipo di rappresentazione che tiene conto di relazioni e dipendenze tra le parole è un ottimo input per una RNN.
<br><br>
La classe Embedding di Keras ha due parametri:
 - 1: Il numero di parole nel nostro dizionario
 - 2: Il numero di embedding vectors da generare
 
 
Per aggiungere lo strato ricorrente alla nostra rete possiamo usare la classe SimpleRNN.
 

In [25]:
from keras.layers import LSTM, SimpleRNN, Embedding

model = Sequential()

model.add(Embedding(num_words, 32))
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, None, 10)          100000    
_________________________________________________________________
simple_rnn_6 (SimpleRNN)     (None, 10)                210       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
Total params: 100,221
Trainable params: 100,221
Non-trainable params: 0
_________________________________________________________________


Compiliamo il modello ed eseguiamo l'addestramento per sole 10 epoche.

In [26]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=512, validation_split=0.2, epochs=10)
model.evaluate(X_test, y_test)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 3s 160us/step - loss: 0.6913 - acc: 0.5274 - val_loss: 0.6869 - val_acc: 0.5692
Epoch 2/10
20000/20000 [==============================] - 2s 117us/step - loss: 0.6721 - acc: 0.6280 - val_loss: 0.6806 - val_acc: 0.5602
Epoch 3/10
20000/20000 [==============================] - 2s 118us/step - loss: 0.6273 - acc: 0.6930 - val_loss: 0.6766 - val_acc: 0.5704
Epoch 4/10
20000/20000 [==============================] - 2s 115us/step - loss: 0.5388 - acc: 0.7816 - val_loss: 0.6582 - val_acc: 0.6194
Epoch 5/10
20000/20000 [==============================] - 2s 119us/step - loss: 0.4305 - acc: 0.8557 - val_loss: 0.6072 - val_acc: 0.6796
Epoch 6/10
20000/20000 [==============================] - 2s 119us/step - loss: 0.3377 - acc: 0.9111 - val_loss: 0.5972 - val_acc: 0.6912
Epoch 7/10
20000/20000 [==============================] - 2s 119us/step - loss: 0.2619 - acc: 0.9419 - val_loss: 0.5759 - v

I risultati della nostra rette ricorrente sono scarsi se confrontati a quelli che abbiamo ottenuto in precedenza con un semplice percettrone multistrato. Inoltre  il modello soffre di un problema di un evidente problema di overfitting sul set di addestramento.
<br>
Le reti ricorrenti soffrono del problema della **scomparsa del gradiente** tra le diverse esecuzioni della rete e questo le rende inadatte per sequenze di dati troppo lunghe.
<br><br>
In questi casi la soluzione migliore è utilizzare le **Long short-term memory** network.